In [0]:
# Importing the main modules
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
# df1 = spark.read.csv("/FileStore/YOURdirectory/Churn.csv" ,inferSchema= True ,header = True , sep = ";")

In [0]:
# df1.write.format("csv").mode("overwrite").option("header", "true").option("inferSchema", True).save("/learn/Churn")

In [0]:
churn = spark.read.format("csv").option("header", True ).option("inferSchema", True).load("/learn/Churn") 

In [0]:
churn.printSchema()
churn.show(5)

root
-- CreditScore: integer (nullable = true)
-- Geography: string (nullable = true)
-- Gender: string (nullable = true)
-- Age: integer (nullable = true)
-- Tenure: integer (nullable = true)
-- Balance: integer (nullable = true)
-- NumOfProducts: integer (nullable = true)
-- HasCrCard: integer (nullable = true)
-- IsActiveMember: integer (nullable = true)
-- EstimatedSalary: integer (nullable = true)
-- Exited: integer (nullable = true)

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
 619| France|Female| 42| 2| 0| 1| 1| 1| 10134888| 1|
 608| Spain|Female| 41| 1| 8380786| 1| 0| 1| 11254258| 0|
 502| France|Female| 42| 8| 1596608| 3| 1| 0| 11393157| 1|
 699| France|Female| 39| 1| 0| 2| 0| 0| 9382663| 0|
 850| Spain|Female| 43| 2|12551082| 1| 1| 1| 790841| 0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 5 rows

In [0]:
# The left columns will be considered independent variables 
formula = RFormula(formula = "Exited ~ ." , featuresCol="features" , labelCol="label", handleInvalid="skip")

In [0]:
# Fitting and transforming our formula object
churn_transformed = formula.fit(churn).transform(churn).select("features","label")

In [0]:
churn_transformed.show(truncate=False)

+----------------------------------------------------------------+-----+
features |label|
+----------------------------------------------------------------+-----+
[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7] |1.0 |
[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7] |0.0 |
[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7] |1.0 |
(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0]) |0.0 |
[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0] |0.0 |
[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0 |
[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0] |0.0 |
[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0 |
[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0] |0.0 |
[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0] |0.0 |
[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.0,8018112.0] |0.0 |
[497.0,0.0,0.0,1.0,24.0,3.0,0.0,2.0,1.0,0.0,7639001.0] |0.0 |
[476.0,1.0,0.0,0.0,34.0,10.0,0.0,2.0,1.0,0.0,2626098.0] |0.0 |
(11,[0,1,4,5,7,10],[549.0,1.0,25.0,5.0,2.0,1.9085779E7]) |0.0 |
[635.0,0.0,0.0,0.0,35.0,7.0,0.0,2.0,1.0,1.0,6595165.0] |0.0 |
[616.0,0.0,1.0,1.0,45.0,3.0,1.4312941E7,2.0,0.0,1.0,6432726.0] |0.0 |
[653.0,0.0,1.0,1.0,58.0,1.0,1.3260288E7,1.0,1.0,0.0,509767.0] |1.0 |
[549.0,0.0,0.0,0.0,24.0,9.0,0.0,2.0,1.0,1.0,1440641.0] |0.0 |
(11,[0,3,4,5,7,10],[587.0,1.0,45.0,6.0,1.0,1.5868481E7]) |0.0 |
[726.0,1.0,0.0,0.0,24.0,6.0,0.0,2.0,1.0,1.0,5472403.0] |0.0 |
+----------------------------------------------------------------+-----+
only showing top 20 rows

In [0]:
churn_train , churn_test = churn_transformed.randomSplit([0.7 , 0.3])

In [0]:
print('Amount Trained : {0} and Amount Tested : {1} Total Data : {2}'.format(churn_train.count(),churn_test.count() , churn_transformed.count() ))

Amount Trained : 6998 and Amount Tested : 3002 Total Data : 10000

In [0]:
# Creating our Decision Tree Classifier Object
dec_tree = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [0]:
# Fitting and creating our train model
model = dec_tree.fit(churn_train)

In [0]:
# Let´s predict our model
predict = model.transform(churn_test)

In [0]:
# Let´s see our Predicted Result 
# Probability = > Confidence Probability , Probability Value
predict.show(truncate=False)

+--------------------------------------------------------+-----+--------------+----------------------------------------+----------+
features |label|rawPrediction |probability |prediction|
+--------------------------------------------------------+-----+--------------+----------------------------------------+----------+
(11,[0,1,3,4,7,10],[624.0,1.0,1.0,37.0,2.0,1.1210455E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,3,4,7,10],[668.0,1.0,1.0,46.0,2.0,2938802.0]) |0.0 |[174.0,33.0] |[0.8405797101449275,0.15942028985507245]|0.0 |
(11,[0,1,3,4,7,10],[794.0,1.0,1.0,33.0,2.0,1.7812271E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[418.0,1.0,39.0,2.0,2.0,904171.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[449.0,1.0,21.0,7.0,2.0,1.7574392E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[475.0,1.0,46.0,10.0,2.0,122953.0]) |1.0 |[174.0,33.0] |[0.8405797101449275,0.15942028985507245]|0.0 |
(11,[0,1,4,5,7,10],[480.0,1.0,28.0,6.0,2.0,4813192.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[496.0,1.0,36.0,7.0,2.0,1.0809828E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[499.0,1.0,57.0,1.0,1.0,1.3137238E7])|1.0 |[35.0,218.0] |[0.1383399209486166,0.8616600790513834] |1.0 |
(11,[0,1,4,5,7,10],[543.0,1.0,36.0,3.0,2.0,2601959.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[543.0,1.0,42.0,5.0,2.0,1.0190534E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[545.0,1.0,55.0,5.0,1.0,1003477.0]) |1.0 |[35.0,218.0] |[0.1383399209486166,0.8616600790513834] |1.0 |
(11,[0,1,4,5,7,10],[553.0,1.0,27.0,3.0,2.0,1.5980016E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[557.0,1.0,24.0,4.0,1.0,2051572.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[564.0,1.0,27.0,6.0,1.0,781976.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[569.0,1.0,44.0,4.0,2.0,1.3439478E7])|0.0 |[174.0,33.0] |[0.8405797101449275,0.15942028985507245]|0.0 |
(11,[0,1,4,5,7,10],[570.0,1.0,46.0,3.0,2.0,82046.0]) |0.0 |[174.0,33.0] |[0.8405797101449275,0.15942028985507245]|0.0 |
(11,[0,1,4,5,7,10],[571.0,1.0,47.0,7.0,2.0,1.1236698E7])|0.0 |[174.0,33.0] |[0.8405797101449275,0.15942028985507245]|0.0 |
(11,[0,1,4,5,7,10],[578.0,1.0,40.0,7.0,2.0,1.0223373E7])|0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
(11,[0,1,4,5,7,10],[598.0,1.0,34.0,4.0,2.0,6089426.0]) |0.0 |[4357.0,497.0]|[0.8976102183765966,0.10238978162340338]|0.0 |
+--------------------------------------------------------+-----+--------------+----------------------------------------+----------+
only showing top 20 rows

In [0]:
# Let´s Evaluate our model Prediction using the Metric areaUnderROC
evalue = BinaryClassificationEvaluator (rawPredictionCol="prediction",labelCol="label", metricName="areaUnderROC")

In [0]:
# Evaluating the method
# A good model may have a evaluate value > 70%
areaUnderRoc = evalue.evaluate(predict)
print(areaUnderRoc)

0.7092700424221134